# Obtenção e tratamento de dados

## Obtenção e carregamento de dados  

In [ ]:
#importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import silhouette_score

In [ ]:
#Carregamento de dados limpos da pasta ../data/
df = pd.read_csv('../Data/dataset.csv')

## Informações sobre o dataset

In [ ]:
#Exibição dos dados
df.head()

In [ ]:
#Exibição de informações sobre os dados
df.info()

In [ ]:
#Exibição de estatísticas descritivas
df.describe()

## Ajuste de tipos

In [ ]:
#Matricula
df['dt_matricula'] = pd.to_datetime(df['dt_matricula'])
#Saida
df['dt_saida'] = pd.to_datetime(df['dt_saida'])
#Primeiro login
df['primeiro_login'] = pd.to_datetime(df['primeiro_login'])
#Ultimo login
df['ultimo_login'] = pd.to_datetime(df['ultimo_login'])

#Idade
df['idade'] = df['idade'].astype('int64')
#Total de logins
df['total_logins'] = df['total_logins'].astype('int64')
#Dias desde o primeiro login
df['dias_desde_primeiro_login'] = df['dias_desde_primeiro_login'].astype('int64')
# Quantidade Mestrado
df['qtd_professores_mestrado'] = df['qtd_professores_mestrado'].astype('int64')
# Quantidade Doutorado
df['qtd_professores_doutorado'] = df['qtd_professores_doutorado'].astype('int64')
# Quantidade Especialização
df['qtd_professores_especialista'] = df['qtd_professores_especialista'].astype('int64')
# Quantidade Graduado
df['qtd_professores_graduado'] = df['qtd_professores_graduado'].astype('int64')
# Quantidade de professores Masculino
df['qtd_professores_M'] = df['qtd_professores_M'].astype('int64')
# Quantidade de professores Feminino
df['qtd_professores_F'] = df['qtd_professores_F'].astype('int64')
# Menor idade de um professor
df['idade_min_professores'] = df['idade_min_professores'].astype('int64')
# Maior idade de um professor
df['idade_max_professores'] = df['idade_max_professores'].astype('int64')

#Data de registro para string
df['dt_registro'] = df['dt_registro'].astype(object)
#Data de inauguração do curso
df['dt_inauguracao_curso'] = df['dt_inauguracao_curso'].astype(object)

#Exibição de informações sobre os dados
df.info()
#Exibição de estatísticas descritivas
df.head()


## Tratamento de valores ausentes

In [ ]:
#preenche com o texto "Abandono" os valores nulos da coluna 'categoria_motivo_cancelamento'
df['categoria_motivo_cancelamento'] = df['categoria_motivo_cancelamento'].fillna('Abandono').where(df['evasao'] == 1,'NA')

#Exibe os valores unicos da coluna 'categoria_motivo_cancelamento'
df['categoria_motivo_cancelamento'].unique()

Exibição dos dados

In [ ]:
#Exibe as informações sobre os dados
df.info()
#Exibe as estatísticas descritivas
df.head()

# Aprendizado de Maquina (Clusterização - KMeans + PCA)

## Amostra dos dados

In [ ]:
df.head()

## Remoção de variáveis correlacionadas

In [ ]:
# Exclua as colunas que não serão usadas
df = df.drop(['cd_aluno','nome', 'curso','cidade','titulacao', 'estado', 'semestre_atual','dias_matriculado_semestre','total_logins','qtd_aprovacoes','valor_mensalidade_bruto', 'percentual_desconto','dias_desde_primeiro_login', 'dt_registro', 'dt_inauguracao_curso','dt_matricula', 'dt_saida', 'primeiro_login', 'ultimo_login', 'idade_min_professores', 'idade_max_professores'], axis=1)

## Separação de variaveis categoricas e numericas

In [ ]:
# Defina as colunas numéricas e categóricas
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = ['genero', 'ds_ensino_medio', 'ds_ingresso', 'cd_curso', 'situacao', 'categoria_motivo_cancelamento']

## Normalização e codificação das variaveis

### Normalização das variáveis numericas

In [ ]:
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# Exibe uma amostra dos dados
df.head()

### Codificação das variáveis categoricas

In [ ]:
# Codificando as colunas categóricas
encoder = OneHotEncoder(drop='first')
df_encoded = encoder.fit_transform(df[cat_cols])

# Criando os nomes das colunas
column_names = encoder.get_feature_names_out(cat_cols)

# Criando um dataframe com os dados codificados
df_encoded = pd.DataFrame(df_encoded.toarray(), columns=column_names)

# Concatenando os dataframes
df = pd.concat([df, df_encoded], axis=1)

# Removendo as colunas categóricas originais
df = df.drop(cat_cols, axis=1)

# Exibe as informações sobre os dados
df.info()

# Exibe uma amostra dos dados
df.head(10)

## Remoção de variável alvo

In [ ]:
# Removendo temporariamente a coluna 'evasao' para não influenciar o modelo
df_sem_alvo = df.drop('evasao', axis=1)

#Cria uma copia do dataframe principal para ser usado no modelo
df_modelo = df.copy()

## Redução de dimensionalidade

### Análise da variância explicada

In [ ]:
# Ajuste do PCA com todos os componentes
pca = PCA()
pca.fit(df_sem_alvo)

# Cálculo da variação explicada cumulativa
cumulative_explained_variance = np.cumsum(pca.explained_variance_ratio_)

# Plot da variação explicada cumulativa
plt.figure(figsize=(10,6))
plt.plot(range(len(cumulative_explained_variance)), cumulative_explained_variance)
plt.title('Variação Explicada Cumulativa por Número de Componentes Principais')
plt.xlabel('Número de componentes')
plt.ylabel('Variação explicada cumulativa')
plt.show()

### Ajuste do número de componentes

In [ ]:
# Ajuste do PCA
pca = PCA(n_components=10)
df_pca = pca.fit_transform(df_sem_alvo)

# Cria um dataframe com os componentes principais
df_pca = pd.DataFrame(df_pca, columns=[f'PC{i+1}' for i in range(10)])

#Concatena os dados do PCA com o dataframe principal
df = pd.concat([df, df_pca], axis=1)

# Atribui os valores do PCA ao dataframe de modelo
df_modelo = df_pca.copy()

## Definição do número de clusters (Elbow Method)

In [ ]:
# Método do cotovelo#
wcss = []
# Testando de 1 a 10 clusters
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(df)
    wcss.append(kmeans.inertia_)

# Exibindo o método do cotovelo
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o')
plt.title('Método do Cotovelo')
plt.xlabel('Número de clusters')
plt.xticks(range(1, 11))
plt.ylabel('WCSS')
plt.show()

## Ajuste do modelo

In [ ]:
# Ajuste do modelo de clusterização nos dados transformados pelo PCA
kmeans = KMeans(n_clusters=2, n_init=10, random_state=42)
kmeans.fit(df_modelo)

# Adicionando os labels dos clusters ao dataframe
df['cluster'] = kmeans.labels_


## Avaliação do modelo

### Silhouette Score

In [ ]:
# Calculando a pontuação da silhueta
score = silhouette_score(df_modelo, kmeans.labels_)

print('Silhouette Score: ', score)

## Visualização dos clusters

### Visualização dos clusters em 2D

In [ ]:
# Criando uma figura e um eixo
fig, ax = plt.subplots( figsize=(10, 7))

# Criando uma lista de cores para cada cluster
colors=[ '#FFC75F', '#FF9671', '#FF6F91', '#D65DB1', '#845EC2']

# Plotando cada cluster
for i in range(2):
    # Selecionando apenas os pontos de dados neste cluster
    cluster = df[df['cluster'] == i]
    ax.scatter(cluster['PC1'], cluster['PC2'], c=colors[i], label=f'Cluster {i+1}', alpha=0.5)

# Adicionando uma legenda
ax.legend()
plt.title('Visualização dos Clusters de estudantes com PCA')
plt.show()

### Médias dos clusters

In [ ]:
# Calculando a média de cada coluna para cada cluster
cluster_means = df.groupby('cluster').mean()

#Coloca a coluna 'evasao' na primeira posição
col = cluster_means.pop('evasao')
cluster_means.insert(0, 'evasao', col)

# Exibindo a média de cada coluna por cluster
cluster_means.head()